# Modelo Matemática - Árvore de Steiner



    definição do modelo goes here.



## lendo dados

In [2]:
from classes import Network, MulticastGroup
from multicastpacking import MulticastPacking, solver
import reader

#instancen of the problem
file = "../../../MPP_instances/n120/b120_1.brite"

# listofopt = "..."

links = reader.get_network (file)
net = Network (links, nodes = 120)


mgroups = [MulticastGroup (g) for g in reader.get_groups (file) ]

problem = MulticastPacking (net, mgroups)

KSIZE = len(problem.groups)+1
NODES = net.nodes+1

## Definição de Variáveis

In [3]:
def create_variable (m, k, var_x, var_y):
    for link in net.links:
        y=link[0],link[1]
        var_y[y]=m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))
        y=link[1],link[0]
        var_y[y]=m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))

    m.update ()

    for d in problem.groups[k-1].members:
        for link in net.links:
            y=link[0],link[1],d
            var_x[y]=m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))
            y=link[1],link[0],d
            var_x[y]=m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))

    m.update ()

## Restrições de Fluxo

In [4]:
## fluxo 1

#fluxo saindo da fonte sk para o destino d no grupo K

def flow1 (m, k, var_x, var_y):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow1',d
        m.addConstr (
            quicksum ( var_x[x] for x in tuplelist (var_x).select ('*',sk,d) )
            -
            quicksum ( var_x[x] for x in tuplelist (var_x).select (sk,'*',d) )
            == -1,
            name=str(_name)
        )

    m.update ()


#fluxo chegando no destino d no grupo K
def flow2 (m, k, var_x, var_y):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow2',d
        m.addConstr (
            quicksum (
                var_x[x] for x in tuplelist (var_x).select ('*',d,d)
            )
            -
            quicksum (
                var_x[x] for x in tuplelist (var_x).select (d,'*',d)
            )
            == 1,
            name=str(_name)
        )

    m.update ()

# conservação de fluxo
def flow3 (m, k, var_x, var_y):
    for d in problem.groups[k-1].members:
        for j in xrange(1,NODES):
            sk = problem.groups[k-1].source
            _name='flow3',d,j   
            m.addConstr (
                quicksum(
                    var_x[x] for x in tuplelist (var_x).select ('*',j,d) 
                    if x[1] not in [sk, d]
                )
                -
                quicksum(
                    var_x[x] for x in tuplelist (var_x).select (j,'*',d) 
                    if x[0] not in [sk, d]
                )
                == 0,
                name=str(_name)
            )            

    m.update ()

## Marcar aresta usada

In [5]:

def set_edge (m, k, var_x, var_y):
    for d in problem.groups[k-1].members:
        for link in net.links:
            x=link[0],link[1],d
            y=link[0],link[1]
            _name='mark',link[0],link[1],d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )            
            x=link[1],link[0],d
            y=link[1],link[0]
            _name='mark',link[1],link[0],d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )

    m.update ()

## Função Objetivo - Custo

In [6]:

def objective_custo (m, var_y, net):
    m.setObjective (
        quicksum (
            var_y[(y[0],y[1])] * net.links[y][0] for y in net.links.keys ()
        )
        +
        quicksum (
            var_y[(y[1],y[0])] * net.links[y][0] for y in net.links.keys ()
        )
    )

## Função Objetivo = Número de Links

In [7]:

def objective_links (m, var_y):
    m.setObjective (
        quicksum (
            var_y[y] for y in tuplelist(var_y)
        )
    )

## Modelo por Número de Links

In [8]:
from gurobipy import *

m = Model ('Steiner Tree') #definindo objetov Model do gurobi

var_y = {} #variável que marca arestas usadas
var_x = {} #variável que controla o fluxo

k = 1  # escolhendo árvore a ser optimizada

create_variable (m, k, var_x, var_y)
flow1 (m, k, var_x, var_y)
flow2 (m, k, var_x, var_y)
flow3 (m, k, var_x, var_y)
set_edge (m, k, var_x, var_y)

objective_links (m, var_y)

In [9]:
m.optimize ()

Optimize a model with 7826 rows, 6720 columns and 24960 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e+00, 1e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+00]
Found heuristic solution: objective 362
Presolve removed 44 rows and 18 columns
Presolve time: 0.07s
Presolved: 7782 rows, 6702 columns, 24900 nonzeros
Variable types: 0 continuous, 6702 integer (6702 binary)

Root relaxation: objective 2.200000e+01, 800 iterations, 0.06 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      22.0000000   22.00000  0.00%     -    0s

Explored 0 nodes (800 simplex iterations) in 0.32 seconds
Thread count was 2 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 2.200000000000e+01, best bound 2.200000000000e+01, gap 0.0%


### Código para Gerar dot da árvore

In [13]:
from graphviz import Digraph

g = Digraph ()

kid = k

#style of nodes (sk, Dk)
for d in problem.groups[kid-1].members:
    g.node (str (d), style='filled', fillcolor='green')

sk = problem.groups[kid-1].source
g.node (str(sk), style='filled', fillcolor='blue')

cost = 0.0
for i in var_y:
    name=var_y[i].getAttr ('varname')
    v = m.getVarByName (str(i))    
    l = i if i in net.links else (i[1],i[0]) #creating the right link to access properties
    if v.getAttr('X') == 1:
        g.edge ( str(i[0]) ,str(i[1]), str(net.properties(l)[0]) )
        cost = cost + net.properties(l)[0]

print (g.source)

digraph {
	114 [fillcolor=green style=filled]
	28 [fillcolor=green style=filled]
	59 [fillcolor=green style=filled]
	116 [fillcolor=green style=filled]
	89 [fillcolor=green style=filled]
	65 [fillcolor=green style=filled]
	88 [fillcolor=green style=filled]
	92 [fillcolor=green style=filled]
	78 [fillcolor=green style=filled]
	37 [fillcolor=green style=filled]
	97 [fillcolor=green style=filled]
	61 [fillcolor=green style=filled]
	19 [fillcolor=green style=filled]
	34 [fillcolor=blue style=filled]
		24 -> 28 [label=597]
		16 -> 88 [label=798]
		22 -> 61 [label=600]
		22 -> 9 [label=70]
		9 -> 65 [label=204]
		9 -> 44 [label=543]
		44 -> 97 [label=169]
		24 -> 22 [label=215]
		22 -> 110 [label=448]
		37 -> 24 [label=196]
		67 -> 92 [label=102]
		9 -> 78 [label=351]
		59 -> 67 [label=413]
		75 -> 89 [label=443]
		9 -> 75 [label=599]
		16 -> 37 [label=708]
		16 -> 23 [label=439]
		34 -> 16 [label=289]
		23 -> 59 [label=346]
		24 -> 19 [label=87]
		24 -> 116 [label=552]
		110 -> 114 [label=1

## Modelo por Custo

In [16]:
from gurobipy import *

m2 = Model ('Steiner Tree') #definindo objetov Model do gurobi

var_y_ = {} #variável que marca arestas usadas
var_x_ = {} #variável que controla o fluxo

k = 1  # escolhendo árvore a ser optimizada

create_variable (m2, k, var_x_, var_y_)
flow1 (m2, k, var_x_, var_y_)
flow2 (m2, k, var_x_, var_y_)
flow3 (m2, k, var_x_, var_y_)
set_edge (m2, k, var_x_, var_y_)

objective_custo (m2, var_y_, net)

In [17]:
gm2.optimize ()

Optimize a model with 7826 rows, 6720 columns and 24960 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+01, 1e+03]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+00]
Found heuristic solution: objective 126495
Presolve removed 44 rows and 18 columns
Presolve time: 0.04s
Presolved: 7782 rows, 6702 columns, 24900 nonzeros
Variable types: 0 continuous, 6702 integer (6702 binary)

Root relaxation: objective 7.026000e+03, 1901 iterations, 0.07 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    7026.0000000 7026.00000  0.00%     -    0s

Explored 0 nodes (1901 simplex iterations) in 0.12 seconds
Thread count was 2 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 7.026000000000e+03, best bound 7.026000000000e+03, gap 0.0%


### Código para Gerar dot da árvore

In [19]:
from graphviz import Digraph

g = Digraph ()

kid = k

#style of nodes (sk, Dk)
for d in problem.groups[kid-1].members:
    g.node (str (d), style='filled', fillcolor='green')

sk = problem.groups[kid-1].source
g.node (str(sk), style='filled', fillcolor='blue')

for i in var_y_:
    name=var_y_[i].getAttr ('varname')
    v = m2.getVarByName (str(i))    
    l = i if i in net.links else (i[1],i[0]) #creating the right link to access properties
    if v.getAttr('X') == 1:
        g.edge ( str(i[0]) ,str(i[1]), str(net.properties(l)[0]) )
        

print g.source

digraph {
	114 [fillcolor=green style=filled]
	28 [fillcolor=green style=filled]
	59 [fillcolor=green style=filled]
	116 [fillcolor=green style=filled]
	89 [fillcolor=green style=filled]
	65 [fillcolor=green style=filled]
	88 [fillcolor=green style=filled]
	92 [fillcolor=green style=filled]
	78 [fillcolor=green style=filled]
	37 [fillcolor=green style=filled]
	97 [fillcolor=green style=filled]
	61 [fillcolor=green style=filled]
	19 [fillcolor=green style=filled]
	34 [fillcolor=blue style=filled]
		67 -> 59 [label=413]
		10 -> 5 [label=154]
		117 -> 110 [label=155]
		65 -> 9 [label=204]
		34 -> 1 [label=262]
		22 -> 61 [label=600]
		1 -> 10 [label=123]
		32 -> 88 [label=863]
		24 -> 37 [label=196]
		8 -> 47 [label=127]
		1 -> 65 [label=85]
		9 -> 32 [label=203]
		1 -> 8 [label=251]
		67 -> 92 [label=102]
		9 -> 78 [label=351]
		8 -> 15 [label=255]
		38 -> 89 [label=382]
		5 -> 67 [label=115]
		15 -> 28 [label=282]
		10 -> 38 [label=380]
		24 -> 19 [label=87]
		24 -> 116 [label=552]
		9 

## Funções Utilitárias

In [37]:
def custo_arvore (model, var, network):
    cost = 0.0
    for i in var:
        name=var[i].getAttr ('varname')
        v = model.getVarByName (str(i))    
        l = i if i in network.links else (i[1],i[0]) #creating the right link to access properties
        if v.getAttr('X') == 1:            
            cost = cost + net.properties(l)[0]
    return cost


def get_num_links (model, var):
    numlinks = 0.0
    for i in var:
        name=var[i].getAttr ('varname')
        v = model.getVarByName (str(i))        
        if v.getAttr('X') == 1:            
            numlinks = numlinks + 1
    return numlinks
        

## Obter todas as soluções - Número de Links

In [39]:
from gurobipy import *

sol1 = {}
sol1_links = {}

for k in xrange (1,KSIZE): 
    M = Model ('Steiner Tree') 
    varY = {} 
    varX = {} 
    create_variable (M, k, varX, varY)
    flow1 (M, k, varX, varY)
    flow2 (M, k, varX, varY)
    flow3 (M, k, varX, varY)
    set_edge (M, k,varX, varY)
    objective_links (M, varY)
    
    M.setParam (GRB.Param.OutputFlag, 0)
    M.optimize ()
    
    sol1[k] = custo_arvore (M, varY, net)
    sol1_links[k] = M.ObjVal

## Obter todas as soluções - Custo

In [40]:
from gurobipy import *

sol2 = {}
sol2_links = {}

for k in xrange (1,KSIZE): 
    M = Model ('Steiner Tree') 
    varY = {} 
    varX = {} 
    create_variable (M, k, varX, varY)
    flow1 (M, k, varX, varY)
    flow2 (M, k, varX, varY)
    flow3 (M, k, varX, varY)
    set_edge (M, k,varX, varY)
    objective_custo (M, varY, net)
    
    M.setParam (GRB.Param.OutputFlag, 0)
    M.optimize ()
    
    sol2[k] = M.ObjVal
    sol2_links[k] = get_num_links (M, varY)   

## Observações

No exemplos há duas situações distintas: no primeiro caso as árvores construídas consideram o número de mínimo de arestas que contectam os participantes do grupo multicast. Por outro lado, no segundo caso se tem árvore com custo menor. Possívelmente, o segundo caso, é possível que o número de arestas seja maior.

Aqui é um bom indício de aplicação do modelo que otimiza custo e Z com limite de hop.


In [36]:
links = 8280.0 + 9269.0 + 10416.0 + 8826.0 + 10889.0
cost = 7026.0 + 6805.0 + 7498.0 + 7364.0 + 8145.0

print 'Custo das árvores por nº de links %s e por otimização de custo %s ' % (links, cost)
print 'Links - Cost: %s' % (links-cost)

Custo das árvores por nº de links 47680.0 e por otimização de custo 36838.0 
Links - Cost: 10842.0


In [45]:
print 'Solução 1 considerando número de Links (Custo): %s ' % sol1
print 'Número de Links da Solução 1: %s' % sol1_links
print 'Solução 2 considerando número de Links (Custo): %s ' % sol2
print 'Número de Links da Solução 2: %s' % sol2_links

Solução 1 considerando número de Links (Custo): {1: 8280.0, 2: 9269.0, 3: 10416.0, 4: 8826.0, 5: 10889.0} 
Número de Links da Solução 1: {1: 22.0, 2: 25.0, 3: 27.0, 4: 26.0, 5: 29.0}
Solução 2 considerando número de Links (Custo): {1: 7026.0, 2: 6805.0, 3: 7498.0, 4: 7364.0, 5: 8145.0} 
Número de Links da Solução 2: {1: 27.0, 2: 30.0, 3: 33.0, 4: 29.0, 5: 36.0}


In [105]:
Sol1_Custo = sum (sol1.values())
Sol2_Custo = sum (sol2.values())
diff = (Sol2_Custo*100.0)/Sol1_Custo

print 'Economia do modelo de custo: %s' %(100.0 - diff)


l = [x * 100 for x in sol2_links.values ()]
diff = [l[i-1]/sol1_links[i] for i in xrange(1,len(l)+1)  ]
diff = [x - 100.0 for x in diff]
diff = map (lambda x: float('%0.2f' % x), diff)

print 'Diferença percentual entre de tamanho das árvores: %s' % diff


Economia do modelo de custo: 22.7390939597
Diferença percentual entre de tamanho das árvores: [22.73, 20.0, 22.22, 11.54, 24.14]
